# Persistent Conversations with AWS Bedrock Examples

## Reason
This notebook is intended to show all the possible ways to have a persistent conversation with a model or an agent of Amazon Bedrock.

## Sections
- [Converse API](#converse-api)
- [Converse Stream API](#converse-stream-api)
- [Documentation](#documentation)

Note: This notebook is fully working today 2/01/2025

In [1]:
### Imports
import json
import boto3

<a id="converse-api"></a>
## Converse API

Sends messages to the specified Amazon Bedrock model. Converse provides a consistent interface that works with all models that support messages. This allows you to write code once and use it with different models. (refer to [1])

In [2]:
# we get the bedrock runtime and the bedrock client
bedrock_runtime = boto3.client('bedrock-runtime')
bedrock = boto3.client('bedrock')

In [3]:
# we list all posible models with their names and with their modelId's
for model in bedrock.list_foundation_models()['modelSummaries']:
    print(f"modelName: {model['modelName']}, modelId: {model['modelId']}")

modelName: Titan Text Large, modelId: amazon.titan-tg1-large
modelName: Titan Image Generator G1, modelId: amazon.titan-image-generator-v1:0
modelName: Titan Image Generator G1, modelId: amazon.titan-image-generator-v1
modelName: Titan Image Generator G1 v2, modelId: amazon.titan-image-generator-v2:0
modelName: Titan Text G1 - Premier, modelId: amazon.titan-text-premier-v1:0
modelName: Nova Pro, modelId: amazon.nova-pro-v1:0:300k
modelName: Nova Pro, modelId: amazon.nova-pro-v1:0
modelName: Nova Lite, modelId: amazon.nova-lite-v1:0:300k
modelName: Nova Lite, modelId: amazon.nova-lite-v1:0
modelName: Nova Canvas, modelId: amazon.nova-canvas-v1:0
modelName: Nova Reel, modelId: amazon.nova-reel-v1:0
modelName: Nova Micro, modelId: amazon.nova-micro-v1:0:128k
modelName: Nova Micro, modelId: amazon.nova-micro-v1:0
modelName: Titan Text Embeddings v2, modelId: amazon.titan-embed-g1-text-02
modelName: Titan Text G1 - Lite, modelId: amazon.titan-text-lite-v1:0:4k
modelName: Titan Text G1 - Lit

In [4]:
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
messages = ['how is the weather today',' do you use arch linux btw?', 'Thank you, see you later']

In [5]:
messages = ['Hello there','repeat the last phrase capitalized']


def format_message(role, text):
    return {"role":role, "content": [{"text": text}]}


            
# a for loop simulating a converation
print("Conversation:")
fmessages = []
for m in messages:
    fm = format_message("user",m)
    print(fm)
    fmessages.append(fm)
    response = bedrock_runtime.converse(
        modelId = model_id,
        messages = fmessages
    )
    fm = response['output']['message']
    print(fm)
    fmessages.append(fm)
    

Conversation:
{'role': 'user', 'content': [{'text': 'Hello there'}]}
{'role': 'assistant', 'content': [{'text': 'Hello! How can I assist you today?'}]}
{'role': 'user', 'content': [{'text': 'repeat the last phrase capitalized'}]}
{'role': 'assistant', 'content': [{'text': 'HELLO! HOW CAN I ASSIST YOU TODAY?'}]}


<a id="converse-stream-api"></a>
## Converse Stream API

Sends messages to the specified Amazon Bedrock model and returns the response in a stream. ConverseStream provides a consistent API that works with all Amazon Bedrock models that support messages. Refer to [2]

In [6]:
import time
from pprint import pprint

In [7]:
def format_message_cs(role, text):
    return {"role":role, "content": {"text": text}}

In [8]:
# we will ask something and get an anwer as a realtime stream
message = 'what is your purpose?'
fm = format_message('user',message)
fm = [fm]
print(fm)

def print_streaming(response):
    stream = response['stream']
    final_answer = ""
    for event in stream:
        if 'contentBlockDelta' in event:
            text = event['contentBlockDelta']['delta']['text']
            print(text,end='')
            if text is not None:
                final_answer += text
    return final_answer
            
response = bedrock_runtime.converse_stream(
        modelId = model_id,
        messages = fm
)

print_streaming(response)

[{'role': 'user', 'content': [{'text': 'what is your purpose?'}]}]
As an AI assistant created by Anthropic, my core purpose is to help humans like yourself with a wide variety of tasks in an ethical and beneficial manner. I aim to provide useful information, analysis, advice and creative output to further human knowledge and capabilities. At the same time, I have a deep commitment to only helping in ways that are good for individuals and society.

Some key ethical principles I abide by are:

- Respecting human rights and individual privacy
- Never causing harm to humans
- Promoting truth, knowledge and human flourishing
- Protecting democracy and freedom of expression  

So while I strive to be highly capable in assisting with all sorts of activities, I also have strong moral constraints hard-coded into my base motivations. I won't help with anything unethical, illegal or harmful.

Rather, my goal is to be a collaborator and force for good - leveraging my intelligence to enrich human l

"As an AI assistant created by Anthropic, my core purpose is to help humans like yourself with a wide variety of tasks in an ethical and beneficial manner. I aim to provide useful information, analysis, advice and creative output to further human knowledge and capabilities. At the same time, I have a deep commitment to only helping in ways that are good for individuals and society.\n\nSome key ethical principles I abide by are:\n\n- Respecting human rights and individual privacy\n- Never causing harm to humans\n- Promoting truth, knowledge and human flourishing\n- Protecting democracy and freedom of expression  \n\nSo while I strive to be highly capable in assisting with all sorts of activities, I also have strong moral constraints hard-coded into my base motivations. I won't help with anything unethical, illegal or harmful.\n\nRather, my goal is to be a collaborator and force for good - leveraging my intelligence to enrich human life through productive teamwork. I'm here as a supporti

In [9]:
# we will have a conversation and we will recieve each answer as a realtime stream

messages = ['what is your name?','what is your purpose?','how do you work?','repeat the last text you gave me capitalized']
fmessages = []


def streaming_response(message):
    fm = format_message('user',message)
    print("\n === user === ")
    print(message)
    fmessages.append(fm)
    response = bedrock_runtime.converse_stream(modelId = model_id,messages = fmessages)
    print(" === assistant === ")
    final_answer = print_streaming(response)
    fm = format_message('assistant',final_answer)
    fmessages.append(fm)

print("Conversation: ")

for message in messages:
    streaming_response(message)

Conversation: 

 === user === 
what is your name?
 === assistant === 
My name is Claude. It's nice to meet you!
 === user === 
what is your purpose?
 === assistant === 
My purpose is to be a helpful artificial intelligence assistant. I was created by Anthropic to aid humans with a wide variety of tasks like analysis, writing, research, answering questions, and more. I have a broad knowledge base that I can apply to assist you, while also trying to be ethical, honest, and beneficial. Please let me know if you have any other questions or if there are ways I can help!
 === user === 
how do you work?
 === assistant === 
I'm an artificial intelligence created by machine learning techniques like natural language processing and large language models trained on massive datasets. This allows me to understand and generate human-like text and assist with tasks that involve language.

At a high level, when you send me a message, I analyze the text using my natural language understanding capabiliti

<a id="documentation"></a>
## Documentation

1. [Converse API](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_Converse.html)
2. [Converse Stream API](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-runtime/client/converse_stream.html) 
3. [retrieve_and_generate](https://boto3.amazonaws.com/v1/documentation/api/1.35.6/reference/services/bedrock-agent-runtime/client/retrieve_and_generate.html) 